In [ ]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import os
import csv
import time
import signal
import schedule

In [ ]:
def search_list(strings, target):
    result = []
    for string in strings:
        if target in string:
            result.append(string)
    return result

In [ ]:
""" class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("Function call timed out") """

In [ ]:
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function timed out")

signal.signal(signal.SIGALRM, timeout_handler)

def itemset(df,thresholds):
    frequent_itemsets = None
    for threshold in thresholds:
        signal.alarm(90)  # set timeout to 60 seconds
        try:
            frequent_itemsets = apriori(df, min_support=threshold, use_colnames=True)
        except TimeoutException:
            print(f"Method did not return results in 1 minute. Stopping and recalling with threshold = {threshold}")
            continue
        signal.alarm(0)  # clear the alarm
        break
        
    return frequent_itemsets

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import os
import csv

p='/Projects'
dirlist = [item for item in os.listdir(p) if os.path.isdir(os.path.join(p, item))]

lenth = len(dirlist)

project='ansible-collection-hardening'
ProjectClone= p + "/" + project
    
try:
            with open(ProjectClone+'/_90_train_Of_Commits.csv','r') as read_obj:
                        csv_reader = csv.reader(read_obj)

                  # convert string to list
                        list_of_files = list(csv_reader)
                        final_list_of_files = [item for sublist in list_of_files for item in sublist]
                        te = TransactionEncoder()
                        te_ary = te.fit(list_of_files).transform(list_of_files)
                        df = pd.DataFrame(te_ary, columns=te.columns_)
                        
                        frequent_itemsets = apriori(df, min_support=0.08, use_colnames=True)
                        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)
                        target = '_(IaC)_'
                        other = '_(Other)_'
                        
                        df = pd.DataFrame(rules)
                        counter = 1
                        df2 = pd.DataFrame()
                        single_antecedent_rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1)]
                        single_consequents_rules= single_antecedent_rules[single_antecedent_rules['consequents'].apply(lambda x: len(x) == 1)]
                        
                        new_rules = []
                        for index, rule in single_consequents_rules.iterrows():

                              antecedents = rule["antecedents"]
                              
                              consequent = rule['consequents']
                              if len(str(antecedents)) != 15 and len(str(consequent)) != 15:  
                                          
                                    new_rules.append(rule)

                        new_rules_df = pd.DataFrame(new_rules)
                        rule_list = new_rules_df.to_dict(orient='records')
                        rule_df = pd.DataFrame(rule_list)
                        rule_df = rule_df.loc[:, ['antecedents', 'consequents', 'support', 'confidence', 'lift']]
                        rule_df.to_csv(p+'/'+project+'/New_IaC_Rules_Apriori.csv')
except FileNotFoundError:
            
            pass
    
